### Import required Python libraries and set plotting parameters 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import time
import os
import os.path
import zipfile
import pandas as pd
from scipy.optimize import curve_fit, least_squares
import sys
import gmsh
import math
import pyvista as pv
import re

from matplotlib.colors import ListedColormap
pv.set_plot_theme("document")

plt.rcParams['figure.figsize'] = [12, 9]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = "DejaVu Serif"
plt.rcParams['font.size'] = 20

from pyvirtualdisplay import Display
display = Display(backend="xvfb", visible=False, size=(800, 600))
display.start()
    
user_name=!whoami # get user name
user_name=user_name[0]
um_view = "/mofem_install/jupyter/%s/um_view" % user_name

### Define utility functions including black-box launch of MoFEM


In [ ]:
class AttrDict(dict):
    def __getattr__(self, attr):
        if attr in self:
            return self[attr]
        raise AttributeError(f"'AttrDict' object has no attribute '{attr}'")
    def __setattr__(self, key, value):
        self[key] = value
        
def replace_template_sdf(params):
    regex = r"\{(.*?)\}"
    with open(params.template_sdf_file) as infile, open(params.sdf_file, 'w') as outfile:
        for line in infile:
            matches = re.finditer(regex, line, re.DOTALL)
            for match in matches:
                for name in match.groups():
                    src = "{" + name + "}"
                    target = str(params[name])
                    line = line.replace(src, target)
            outfile.write(line)

def get_young_modulus(K, G):
    E = 9. * K * G /(3. * K + G)
    return E

def get_poisson_ratio(K, G):
    nu = (3. * K - 2. * G) / 2. / (3. * K + G)
    return nu

def get_bulk_modulus(E, nu):
    K = E / 3. / (1. - 2. * nu)
    return K

def get_shear_modulus(E, nu):
    G = E / 2. / (1. + nu)
    return G

def parse_log_file(filepath):
    force, time, area = [], [], []
    with open(filepath, "r") as log_file:
        for line in log_file:
            line = line.strip()
            if "Contact force:" in line:
                line = line.split()
                time.append(float(line[6]))
                force.append(float(line[10]))
            if "Contact area:" in line:
                line = line.split()
                area.append(float(line[8]))
    return time, force, area

def generate_config(params):
    with open(params.config_file, 'w') as f:
        data = [f"[block_2]", f"id={params.mfront_block_id}", "add=BLOCKSET", f"name=MFRONT_MAT_{params.mfront_block_id}"]
        for line in data:
            f.write(line + '\n')
    return

def mofem_compute_force_indent(params):
    !rm -rf out*
    
    mi_param_2 = 0
    mi_param_3 = 0
    mi_param_4 = 0
    
    if params.material_model == "LinearElasticity":
        mi_block = "LinearElasticity"
        mi_param_0 = params.young_modulus
        mi_param_1 = params.poisson_ratio
    elif params.material_model == "SaintVenantKirchhoffElasticity":
        mi_block = "SaintVenantKirchhoffElasticity"
        mi_param_0 = params.young_modulus
        mi_param_1 = params.poisson_ratio
    elif params.material_model == "NeoHookeanHyperElasticity":
        mi_block = "SignoriniHyperElasticity"
        mi_param_0 = get_bulk_modulus(params.young_modulus, params.poisson_ratio)
        mi_param_1 = 0.5 * get_shear_modulus(params.young_modulus, params.poisson_ratio)
    elif params.material_model == "StandardLinearSolid":
        mi_block = "StandardLinearSolid"
        mi_param_0 = get_bulk_modulus(params.young_modulus_0, params.poisson_ratio_0)
        mi_param_1 = get_shear_modulus(params.young_modulus_0, params.poisson_ratio_0)
        mi_param_2 = get_bulk_modulus(params.young_modulus_1, params.poisson_ratio_1)
        mi_param_3 = get_shear_modulus(params.young_modulus_1, params.poisson_ratio_1)
        mi_param_4 = params.relax_time_1
    else:
        print("Unknown material model: " + params.material_model)
        return
        
    replace_template_sdf(params)
        
    !export OMPI_MCA_btl_vader_single_copy_mechanism=none && \
    nice -n 10 mpirun --oversubscribe --allow-run-as-root \
    -np {params.nproc} {um_view}/tutorials/adv-1/contact_2d \
    -file_name {params.part_file} \
    -sdf_file {params.sdf_file} \
    -order {params.order} \
    -ts_dt {params.time_step} \
    -ts_max_time {params.final_time} \
    -mi_lib_path_{params.mfront_block_id} {um_view}/mfront_interface/libBehaviour.so \
    -mi_block_{params.mfront_block_id} {mi_block} \
    -mi_param_{params.mfront_block_id}_0 {mi_param_0} \
    -mi_param_{params.mfront_block_id}_1 {mi_param_1} \
    -mi_param_{params.mfront_block_id}_2 {mi_param_2} \
    -mi_param_{params.mfront_block_id}_3 {mi_param_3} \
    -mi_param_{params.mfront_block_id}_4 {mi_param_4} \
    -mi_save_volume 1 \
    -mi_save_gauss 0 \
    2>&1 | tee {params.log_file}

    time, react, area = parse_log_file(params.log_file)
    indent = np.asarray(time) * (params.max_indentation / params.final_time)
    force = np.asarray(react)
    
    return indent, force, area

def show_results(params):
    out_to_vtk = !ls -c1 out_*h5m
    last_file=out_to_vtk[0]
    print(last_file)
    !mbconvert {last_file} {last_file[:-3]}vtk
    
    import pyvista as pv
    import matplotlib.pyplot as plt
    from matplotlib.colors import ListedColormap
    import matplotlib.image as mpimg
    import re, os

    mesh = pv.read(last_file[:-3] + "vtk")

    mesh=mesh.warp_by_vector('DISPLACEMENT', factor=1)
    if params.show_edges:
        mesh=mesh.shrink(0.95)
    
    if params.show_field == "DISPLACEMENT" or params.show_field == "displacement":
        field = "DISPLACEMENT"
        if params.show_component == "X" or params.show_component == 'x':
            comp = 0
        elif params.show_component == "Y" or params.show_component == 'y':
            comp = 1
        else:
            print("Wrong component {0} of the field {1}".format(params.show_component, params.show_field))
            return
        
    if params.show_field == "STRESS" or params.show_field == "stress":
        field = "STRESS"
        if params.show_component == "X" or params.show_component == "x":
            comp = 0
        elif params.show_component == "Y" or params.show_component == "y":
            comp = 4
        elif params.show_component == "XY" or params.show_component == "xy":
            comp = 1
        else:
            print("Wrong component {0} of the field {1}".format(params.show_component, params.show_field))
            return

    p = pv.Plotter(notebook=True)
    p.add_mesh(mesh, scalars=field, component=comp, show_edges=True, smooth_shading=False, cmap="turbo")
    
    circle = pv.Circle(radius=params.indenter_radius, resolution=1000)
    circle = circle.translate((0, params.indenter_radius - params.max_indentation, 0), inplace=False)
    p.add_mesh(circle, color="grey")
    
    p.camera_position = "xy"
    p.show(jupyter_backend='ipygany')


def generate_mesh(params):
    gmsh.initialize()
    gmsh.model.add("Nanoindentation")
    
    a = params.refine_radius    
    H = params.mesh_height 
    L = params.mesh_length
    R = params.indenter_radius
    
    # Creating points
    tol = 1e-3
    
    print(a, H, R)
    
    if a < H / 2 and H > R:
        point1 = gmsh.model.geo.addPoint(0, 0, 0, tol)
        point2 = gmsh.model.geo.addPoint(0, -a, 0, tol)
        point3 = gmsh.model.geo.addPoint(a, 0, 0, tol)
        point4 = gmsh.model.geo.addPoint(0, -H, 0, tol)
        point5 = gmsh.model.geo.addPoint(L, -H, 0, tol)
        point6 = gmsh.model.geo.addPoint(L, 0, 0, tol)

        # Creating connection lines
        arc1 = gmsh.model.geo.addCircleArc(point3, point1, point2)
        line1 = gmsh.model.geo.addLine(point1, point2)
        line2 = gmsh.model.geo.addLine(point2, point4)
        line3 = gmsh.model.geo.addLine(point4, point5)
        line4 = gmsh.model.geo.addLine(point5, point6)
        line5 = gmsh.model.geo.addLine(point6, point3)
        line6 = gmsh.model.geo.addLine(point3, point1)

        loop1 = gmsh.model.geo.addCurveLoop([line1, -arc1, line6])
        surface1 = gmsh.model.geo.addPlaneSurface([loop1])

        loop2 = gmsh.model.geo.addCurveLoop([arc1, line2, line3, line4, line5])
        surface2 = gmsh.model.geo.addPlaneSurface([loop2])

        # This command is mandatory and synchronize CAD with GMSH Model. The less you launch it, the better it is for performance purpose
        gmsh.model.geo.synchronize()

        domain = gmsh.model.addPhysicalGroup(2, [surface1, surface2])
        gmsh.model.setPhysicalName(2, domain, '!_DOMAIN')
        contact = gmsh.model.addPhysicalGroup(1, [line5, line6])
        gmsh.model.setPhysicalName(1, contact, 'CONTACT')
        fix_x = gmsh.model.addPhysicalGroup(1, [line1, line2])
        gmsh.model.setPhysicalName(1, fix_x, 'FIX_X')
        fix_y = gmsh.model.addPhysicalGroup(1, [line3])
        gmsh.model.setPhysicalName(1, fix_y, 'FIX_Y')
        gmsh.model.mesh.setSize(gmsh.model.getEntitiesInBoundingBox(0, -H, 0, L, 0, 0), params.far_field_size)
        gmsh.model.mesh.setSize(gmsh.model.getEntitiesInBoundingBox(0, -a, 0, a, 0, 0), params.near_field_size)
    else:
        point1 = gmsh.model.geo.addPoint(0, 0, 0, tol)
        point2 = gmsh.model.geo.addPoint(0, -H, 0, tol)
        point3 = gmsh.model.geo.addPoint(H, -H, 0, tol)
        point4 = gmsh.model.geo.addPoint(H, 0, 0, tol)
        point5 = gmsh.model.geo.addPoint(L, -H, 0, tol)
        point6 = gmsh.model.geo.addPoint(L, 0, 0, tol)

        # Creating connection lines
        line1 = gmsh.model.geo.addLine(point1, point2)
        line2 = gmsh.model.geo.addLine(point2, point3)
        line3 = gmsh.model.geo.addLine(point3, point4)
        line4 = gmsh.model.geo.addLine(point4, point1)
        line5 = gmsh.model.geo.addLine(point3, point5)
        line6 = gmsh.model.geo.addLine(point5, point6)
        line7 = gmsh.model.geo.addLine(point6, point4)

        loop1 = gmsh.model.geo.addCurveLoop([line1, line2, line3, line4])
        surface1 = gmsh.model.geo.addPlaneSurface([loop1])

        loop2 = gmsh.model.geo.addCurveLoop([-line3, line5, line6, line7])
        surface2 = gmsh.model.geo.addPlaneSurface([loop2])

        # This command is mandatory and synchronize CAD with GMSH Model. The less you launch it, the better it is for performance purpose
        gmsh.model.geo.synchronize()

        domain = gmsh.model.addPhysicalGroup(2, [surface1, surface2])
        gmsh.model.setPhysicalName(2, domain, '!_DOMAIN')
        contact = gmsh.model.addPhysicalGroup(1, [line7, line4])
        gmsh.model.setPhysicalName(1, contact, 'CONTACT')
        fix_x = gmsh.model.addPhysicalGroup(1, [line1])
        gmsh.model.setPhysicalName(1, fix_x, 'FIX_X')
        fix_y = gmsh.model.addPhysicalGroup(1, [line2, line5])
        gmsh.model.setPhysicalName(1, fix_y, 'FIX_Y1')
        
        gmsh.model.mesh.setSize(gmsh.model.getEntitiesInBoundingBox(0, -H, 0, L, 0, 0), params.far_field_size)
        gmsh.model.mesh.setSize(gmsh.model.getEntitiesInBoundingBox(0, -H, 0, H, 0, 0), params.near_field_size)
        
    gmsh.model.mesh.generate(2)    

    # Save mesh
    gmsh.write(params.med_file)

    # Finalize GMSH = END OF CODE=
    gmsh.finalize()
    
    generate_config(params)
    
    !read_med -med_file {params.med_file} -output_file {params.mesh_file} -meshsets_config {params.config_file} -log_sl inform
    
    params.part_file = os.path.splitext(params.mesh_file)[0] + "_" + str(params.nproc) + "p.h5m"
    #partition the mesh into nproc parts
    !{um_view}/bin/mofem_part \
    -my_file {params.mesh_file} \
    -my_nparts {params.nproc} \
    -output_file {params.part_file} \
    -dim 2 -adj_dim 1
    
    if params.show_mesh:
        !mbconvert {params.mesh_file} {params.vtk_file}

        mesh = pv.read(params.vtk_file )
        mesh = mesh.shrink(0.95)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        circle = pv.Circle(radius=params.indenter_radius, resolution=1000)
        circle = circle.translate((0, params.indenter_radius, 0), inplace=False)

        p.add_mesh(circle, color="grey")
        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
    
    return

def hertz_press(indent, params):   
    Es = params.young_modulus / (1 - params.poisson_ratio**2)    
    return 4./3. * Es * np.sqrt(params.indenter_radius) * pow(indent, 3./2.)

def hertz_area(indent, params):   
    return np.pi * indent * params.indenter_radius

def residual_mofem(var, params): 
    
    params.young_modulus = var[0]
    params.poisson_ratio = 0.49
    
    indent, force, area = mofem_compute_force_indent(params)
    
    res = (np.asarray(force)[1:] - np.asarray(params.force_exp)) / np.asarray(params.error_exp)
    
    ls = 0.5 * np.linalg.norm(res) **2
    
    print("LS:", ls)
    print("E: ", var[0])
    
    return res

def hertz(indent, E): 
    """Plot force-indentation curve using Hertz formula
    
    Parameters
    ----------
    Es : float
        effective elastic modulus, Es = E / (1 - nu**2)    
    """
    
    nu = 0.49
    Es = E / (1 - nu**2)
    
    return 4./3. * Es * np.sqrt(params.indenter_radius) * pow(indent, 3./2.)

### Sketch of the problem setup

<!-- ![indent.png](attachment:indent.png) -->

<div>
<img src="attachment:indent.png" width="400px"/>
</div>

### Set simulation parameters

In [ ]:
params = AttrDict()

params.med_file = "mesh_2d.med"
params.mesh_file = "mesh_2d.h5m"
params.vtk_file = "mesh_2d.vtk"

params.load_hist = "load.txt"
params.log_file = "log_indent"

params.template_sdf_file = "template_sdf.py"
params.sdf_file = "sdf.py"

params.config_file = "bc.cfg"
params.mfront_block_id = 10

params.nproc = 8 # number of processors/cores used
params.order = 2 #order of approximation functions

params.final_time = 1
params.time_step = 0.02

params.indenter_radius = 3
params.max_indentation = 1
    
params.refine_radius = np.sqrt(params.indenter_radius * params.max_indentation) # a_hertz = sqrt(R * d)
  
params.mesh_length = params.refine_radius * 40
params.mesh_height = params.mesh_length * 2

params.far_field_size = params.mesh_height / 10
params.near_field_size = params.refine_radius / 10

### Generate and visualise the mesh

In [ ]:
params.show_mesh = True
generate_mesh(params)

### Read experimental data

In [ ]:
data = pd.read_csv("Avg_F_ind_RoV.tsv",delimiter = "\t", skiprows=4)
data = data.loc[(data['Avg Indentation [nm] '] < 1000) & (data['Avg Indentation [nm] '] > 0)] # Consider first 1 um

indent = data["Avg Indentation [nm] "] / 1e3 #um
force = data[" Avg Force [nN] "] #nN
error = data[" Error Force [nN] "] #nN

popt, pcov = curve_fit(hertz, indent, force, sigma=error)
E = popt[0]
print("Hertz fitting, E =", E)

plt.plot(indent/params.indenter_radius, force, label="Experiment", lw=2, ls='--')
plt.fill_between(indent/params.indenter_radius, force - 0.6745 * error, force + 0.6745 * error, facecolor='blue', alpha=0.25)
plt.plot(indent/params.indenter_radius, hertz(indent, popt[0]), label="Hertz theory", lw=2)

plt.xlabel("Normalised indentation, δ/R")
plt.ylabel("Force, nN")
plt.legend(loc='upper left')
plt.grid()


### Run parameter identification

In [ ]:
load_step = 20
data_step = round(data.shape[0] / load_step)
                  
data = data[data_step-1::data_step]

indent = data["Avg Indentation [nm] "] / 1e3 #um
force = data[" Avg Force [nN] "] #nN
error = data[" Error Force [nN] "] #nN

params.max_indent = max(indent)
params.time_step = 1. / (indent.shape[0])

print(force.shape)

params.indent_exp = indent
params.force_exp = force
params.error_exp = error

E0 = [0.5] 

params["material_model"] = "NeoHookeanHyperElasticity"
minimum = optimize.least_squares(residual_mofem, x0=E0, args=(params,), method='lm',
                                 diff_step=1e-3, xtol=1e-3, ftol=1e-3)
E = minimum.x[0]
print("MoFEM fitting, E =", E)

### Run simulation with identified value

In [ ]:
params["young_modulus"] = minimum.x[0]
params["poisson_ratio"] = 0.49
params["material_model"] = "NeoHookeanHyperElasticity"
indent_1, force_1, area_1 = mofem_compute_force_indent(params)

### Compare MoFEM and Hertz fitting

In [ ]:
data = pd.read_csv("Avg_F_ind_RoV.tsv",delimiter = "\t", skiprows=4)
data = data.loc[(data['Avg Indentation [nm] '] < 1000) & (data['Avg Indentation [nm] '] > 0)]

indent = data["Avg Indentation [nm] "] / 1e3 #um
force = data[" Avg Force [nN] "] #nN
error = data[" Error Force [nN] "] #nN

popt, pcov = curve_fit(hertz, indent, force, sigma=error)
E = popt[0]
print("Hertz fitting, E =", E)

plt.plot(indent/params.indenter_radius, force, label="Experiment", lw=2, ls='--')
plt.fill_between(indent/params.indenter_radius, force - 0.6745 * error, force + 0.6745 * error, facecolor='blue', alpha=0.25)
plt.plot(indent_1/params.indenter_radius, force_1, marker='o', label="MoFEM hyperelastic", lw=2)
plt.plot(indent/params.indenter_radius, hertz(indent, popt[0]), label="Hertz theory", lw=2)

plt.xlabel("Normalised indentation, δ/R")
plt.ylabel("Force, nN")
plt.legend(loc='upper left')
plt.grid()

plt.xlabel("Normalised indentation, δ/R")
plt.ylabel("Force, nN")
plt.legend(loc='upper left')
plt.grid()